In [ ]:
# Import necessary libraries
import os
import django
import pandas as pd
import json

In [ ]:
# Set up Django environment
os.chdir("..")
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'gymcrowd.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true" 
django.setup()


In [ ]:
# Import models
from apps.workouts.models import Exercise


In [ ]:
# Load JSON data into a pandas DataFrame
json_file_path = 'scripts/exercises.json'

with open(json_file_path, 'r') as f:
    exercise_data = json.load(f)

# Convert JSON data to pandas DataFrame
df = pd.DataFrame(exercise_data)
print(f"Loaded {len(df)} exercises from the JSON file.")
df.head()  # Display the first few rows for verification


In [ ]:
# Transform data for secondary muscles and instructions
df['secondaryMuscles'] = df['secondaryMuscles'].apply(lambda x: ", ".join(x) if isinstance(x, list) else "")
df['instructions'] = df['instructions'].apply(lambda x: " ".join(x) if isinstance(x, list) else "")
df.head()  # Check transformed data


In [ ]:
# Fetch existing exercises from the database
existing_exercises = set(Exercise.objects.values_list('name', flat=True))

# Split data into new and existing records
df_existing = df[df['name'].isin(existing_exercises)]
df_new = df[~df['name'].isin(existing_exercises)]

print(f"Existing exercises: {len(df_existing)}")
print(f"New exercises: {len(df_new)}")


In [ ]:
# Update existing records
for _, row in df_existing.iterrows():
    Exercise.objects.filter(name=row['name']).update(
        body_part=row['bodyPart'],
        equipment=row['equipment'],
        gif_url=row['gifUrl'],
        target=row['target'],
        secondary_muscles=row['secondaryMuscles'],
        instructions=row['instructions'],
    )
print("Updated existing records.")


In [ ]:
# Bulk insert new records
new_records = [
    Exercise(
        name=row['name'],
        body_part=row['bodyPart'],
        equipment=row['equipment'],
        gif_url=row['gifUrl'],
        target=row['target'],
        secondary_muscles=row['secondaryMuscles'],
        instructions=row['instructions'],
    )
    for _, row in df_new.iterrows()
]
Exercise.objects.bulk_create(new_records, batch_size=500)

print(f"Inserted {len(new_records)} new records.")


In [ ]:
# Verify the database content
total_exercises = Exercise.objects.count()
print(f"Total exercises in the database: {total_exercises}")

# Display some records from the database
Exercise.objects.all()[:5]  # Display the first 5 records
